In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
from scipy.interpolate import interp1d

In [309]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)
    
def get_pts_manual(event):
    fig.canvas.draw_idle()
    global ix,iy, pts_manual
    if event.key == 'm':
        plt.plot(ix,iy, 'x', color = 'tab:red') 
        pts_manual.append([ix,iy])


def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())


# def bound_points(event): 
#     fig.canvas.draw_idle()
#     global ix,iy, boundpts
#     if len(boundpts)<4:
#         #print(event.key)
#         if event.key == 'b':
#             boundpts.append([ix,iy])
#             plt.plot(ix,iy, 'gx')

                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    fig.canvas.draw_idle()
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(event.xdata,event.ydata, 'bo') 
        try:
            idx = np.where(((pts[0,:] - event.xdata)**2 + (pts[1,:] - event.ydata)**2)<400)[0]
            pts[:, idx] = 0
        except:
            print('bleh')
            
            
def bound_polygon(event):
    fig.canvas.draw_idle()
    global ix,iy, boundpts
    #print(event.key)
    if event.key == 'b':
        boundpts.append([ix,iy])
        plt.plot(ix,iy, 'gx')

In [ ]:
boundry_x = []
boundry_y = []
for i in range(len(pts)):
    print((i+1)%len(pts))
    f = interp1d([pts[i,1], pts[(i+1)%len(pts),1]], [pts[i,0],pts[(i+1)%len(pts),0]] ,bounds_error=False, fill_value = 'extrapolate')
    h = np.linspace(pts[i,1],pts[(i+1)%len(pts),1], abs(int(pts[(i+1)%len(pts),1]-pts[i,1])))
    boundry_x+=list(f(h))
    boundry_y+=list(h)
    
plt.figure(2)
plt.plot(boundry_x, boundry_y)
plt.plot(pts[:,0], pts[:,1], 'o')

boundry_x = np.array(boundry_x)
roi = np.zeros((img.shape[0],img.shape[1]))
h = np.arange(int(np.min(boundry_y)),int(np.max(boundry_y)))
for i,y in enumerate(h):
    xs = np.where((boundry_y-y)**2<1)[0]
    xmin = int(np.min(boundry_x[xs]))
    xmax = int(np.max(boundry_x[xs]))
    roi[y,xmin:xmax] = 1
    
plt.figure(3)
plt.imshow(roi)
plt.colorbar()

In [215]:
def get_path(pts):
    path = 0
    for i in range(len(pts)):
        path+= np.sqrt((pts[i%len(pts),0]-pts[(i+1)%len(pts),0])**2+ (pts[i%len(pts),1]-pts[(i+1)%len(pts),1])**2)
    return path

In [150]:
ls

3ps_49mW_50att_250kHz.npy    IMG_1833.png
4ps_29mW_50att_250kHz.npy    IMG_1835.png
5ps_0.3mW_250kHz.npy         IMG_1836.png
5ps_0.3mW_250kHz_2.npy       IMG_1837.png
5ps_103mW_250kHz.npy         IMG_1838.png
5ps_17mW_50att_250kHz.npy    IMG_1839.png
5ps_18mW_250kHz.npy          IMG_1840.png
5ps_3.3mW_250kHz.npy         IMG_1841.png
5ps_31mW_250kHz.npy          IMG_1842.png
5ps_45mW_250kHz.npy          IMG_1843.png
5ps_58mW_250kHz.npy          IMG_1844.png
5ps_70mW_250kHz.npy          IMG_1845.png
5ps_8.5mW_250kHz.npy         IMG_1846.png
5ps_8.5mW_250kHz_2.npy       IMG_1847.png
5ps_84mW_250kHz.npy          IMG_1848 2.png
Digitizer.ipynb              IMG_1851.png
Digitizer_clean.ipynb        IMG_1852.png
IMG_1828.png                 IMG_1853.png
IMG_1829.png                 IMG_1854.png
IMG_1830.HEIC                IMG_1855.png
IMG_1830.png                 IMG_1856.png
IMG_1831.png                 Plot_digitized_traces.ipynb
IMG_1832.png                 README.md


In [321]:
filename = 'IMG_1831.png'

In [312]:
#set calpts (enter), boundpts (b), line_col (-)

In [322]:
img=mpimg.imread(filename)
fig = plt.figure(figsize=(10,5))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
#bid = fig.canvas.mpl_connect('key_press_event', bound_points)
bid = fig.canvas.mpl_connect('key_press_event', bound_polygon)
lid = fig.canvas.mpl_connect('key_press_event', line_color)
eid = fig.canvas.mpl_connect('key_press_event', mouse_eraser)
mid = fig.canvas.mpl_connect('key_press_event', get_pts_manual)

ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
boundpts = []
pts_manual = []
imgplot = plt.imshow(img)
plt.show()

In [323]:
save_name = 'test'

1595.2818181818177 612.1545454545453
2098.256366038453 1705.5967745938967
3646.854846716644 1705.5967745938967
1737.3433316985772 1705.5967745938967
1727.1046640577297 776.4376861869825
1801.3350044538743 1641.6051018385997
1803.8946713640862 1531.5394246994886
2397.737394533244 1503.3830886871579
2610.18974808083 1011.9270419264759
2628.1074164523134 970.9723713630858
2643.4654179135846 912.1000324282124
2658.823419374856 968.4127044528739
2717.695758309729 835.3100251218559
2738.1730935914247 766.199018546135
2776.5680972446025 906.9806986077888
2937.8271125879514 1280.6920674987239
3672.451515818763 1488.0250872258866
3659.653181267703 1646.7244356590234
1814.133339004934 1646.7244356590234
2466.848401108965 1209.021394012791
2610.958580894108 1306.3604274475472
-


In [310]:
#roi for 4 boundpts
# mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
# mask2 = np.copy(mask)
# mask2[mask2>0.02] = 0
# X,Y=np.meshgrid(np.arange(mask2.shape[1]),np.arange(mask2.shape[0]))
# bb = np.array(boundpts)
# x0 = np.min(bb[:, 0]) #x
# y0 = np.max(bb[:, 1]) #y
# x1 = np.max(bb[:, 0]) #x
# y1 = np.min(bb[:, 1]) #y
# roi = np.zeros(mask2.shape)
# roi[np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[0],np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[1]] = 1
# plt.figure(1)
# pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
# plt.plot(pts[0,:], pts[1,:], 'x', color = 'palevioletred')
# plt.show()

In [324]:
#roi for polygon
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
mask2 = np.copy(mask)
mask2[mask2>0.02] = 0
boundpts = np.array(boundpts)
boundry_x = []
boundry_y = []
for i in range(len(boundpts)):
    print((i+1)%len(boundpts))
    f = interp1d([boundpts[i,1], boundpts[(i+1)%len(boundpts),1]], [boundpts[i,0],boundpts[(i+1)%len(boundpts),0]] ,bounds_error=False, fill_value = 'extrapolate')
    h = np.linspace(boundpts[i,1],boundpts[(i+1)%len(boundpts),1], abs(int(boundpts[(i+1)%len(boundpts),1]-boundpts[i,1])))
    boundry_x+=list(f(h))
    boundry_y+=list(h)
    
plt.figure(1)
plt.plot(boundry_x, boundry_y, 'g', alpha = 0.4)

boundry_x = np.array(boundry_x)
roi = np.zeros((img.shape[0],img.shape[1]))
h = np.arange(int(np.min(boundry_y)),int(np.max(boundry_y)))
for i,y in enumerate(h):
    xs = np.where((boundry_y-y)**2<1)[0]
    xmin = int(np.min(boundry_x[xs]))
    xmax = int(np.max(boundry_x[xs]))
    roi[y,xmin:xmax] = 1
    
plt.figure(1)
pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
plt.plot(pts[0,:], pts[1,:], 'x', color = 'palevioletred', alpha = 0.4)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
0


In [196]:
#manual correction, erase (space), add (m)
if len(pts_manual) > 0: pts = np.hstack((pts,np.array(pts_manual).T))
pts = np.vstack((pts[0,:][(pts[0,:]>x0)&(pts[1,:]<y0)],pts[1,:][(pts[0,:]>x0)&(pts[1,:]<y0)]))
plt.figure(1)
plt.plot(pts[0,:], pts[1,:], 'gx')
plt.show()

In [198]:
#interpolate and save
calval = [252,260,0.0,1]
fx = interp1d([calpts[0][0],calpts[1][0]], [calval[0],calval[1]],bounds_error=False, fill_value = 'extrapolate')
fy = interp1d([calpts[2][1],calpts[3][1]],[calval[2],calval[3]],bounds_error=False, fill_value = 'extrapolate')
data = {}
data['x'] = fx(pts[0,:])
data['y'] = fy(pts[1,:])
np.save(save_name+'.npy',data)

In [122]:
plt.figure(3)
plt.plot(data['x'],data['y'], 'o')

2077.3222494275747 2123.6978426577625
3465.667970042179 2133.6145978050095
1873.0370933942827 2119.7311405988635
1873.0370933942827 1258.9567938178084
